<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2025_01_17_Configure_Workflow_Actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to configure a webhook action in a Graphlit workflow, and validate success and failure events.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.

Visit https://webhook.site to get a unique Webhook URL for testing.

---

Install Graphlit Python client SDK

In [8]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [9]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [10]:
from typing import List, Optional
from datetime import datetime, timedelta

async def ingest_uri(uri: str, workflow_id: str):
    if graphlit.client is None:
        return;

    try:
        # Using synchronous mode, so the notebook waits for the content to be ingested
        response = await graphlit.client.ingest_uri(uri=uri, workflow=input_types.EntityReferenceInput(id=workflow_id), is_synchronous=True)

        return response.ingest_uri.id if response.ingest_uri is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_workflow(webhook_uri: str):
    if graphlit.client is None:
        return;

    input = input_types.WorkflowInput(
        name="Workflow",
        actions=[input_types.WorkflowActionInput(
            connector=input_types.IntegrationConnectorInput(
                type=enums.IntegrationServiceTypes.WEB_HOOK,
                uri=webhook_uri
            )
        )],
    )

    try:
        response = await graphlit.client.create_workflow(input)

        return response.create_workflow.id if response.create_workflow is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_workflows():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_workflows(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)


Execute Graphlit example

In [11]:
from IPython.display import display, Markdown, HTML
import time
from datetime import datetime

# Remove any existing contents and workflows; only needed for notebook example
await delete_all_workflows()
await delete_all_contents()

print('Deleted all contents and workflows.')

# NOTE: go to https://webhook.site/, and copy your unique URL
webhook_uri = "https://webhook.site/TODO"

uri = "https://graphlitplatform.blob.core.windows.net/test/documents/2404.17723v2.pdf" # FINISHED
#uri = "https://graphlitplatform.blob.core.windows.net/test/documents/bad-document.pdf" # ERRORED: image saved as .pdf

workflow_id = await create_workflow(webhook_uri)

if workflow_id is not None:
    print(f'Created workflow [{workflow_id}].')

    content_id = await ingest_uri(uri, workflow_id)

    if content_id is not None:
        print(f'Ingested content [{content_id}].')


Deleted all contents and workflows.
Created workflow [1d741fed-38d5-4165-8035-20260e803492].
Ingested content [c7e95daf-fd9e-4873-a13d-661595d7e5b4].


Will receive this JSON on errored entity event.

```
{
    "entity": {
        "contentType": "File",
        "c4id": "c42gAzYkp2QxDf5oVEbHSag2VHARYVPRs6Agc3d6sUpY6hwW4KBhBDvhXy93ExFNxykGzHXm5cLiHsjJJKVevfjvQV",
        "fileType": "Document",
        "mimeType": "application/pdf",
        "format": "PDF",
        "formatName": "Adobe Portable Document Format",
        "fileExtension": ".pdf",
        "fileName": "bad-document.pdf",
        "fileSize": 321215,
        "masterUri": "https://graphlit202407156de7e4a2.blob.core.windows.net/files/551d03f8-680f-4a97-8c94-fb62bc55df40/bad-document.pdf",
        "workflowId": "a1c329fc-6568-4028-8276-66e8ccbb70a5",
        "projectId": "6df74633-edfc-4a1e-b49d-a51ed2220caa",
        "name": "bad-document.pdf",
        "uri": "https://graphlitplatform.blob.core.windows.net/test/documents/bad-document.pdf",
        "error": "Failed to handle rendition [https://graphlit202407156de7e4a2.blob.core.windows.net/files/551d03f8-680f-4a97-8c94-fb62bc55df40/bad-document.pdf] metadata indexing.",
        "modifiedDate": {
            "date": "2025-01-18T04:01:53Z",
            "epoch": 1737172913
        },
        "startedDate": {
            "date": "2025-01-18T04:01:50Z",
            "epoch": 1737172910
        },
        "finishedDate": {
            "date": "2025-01-18T04:01:53Z",
            "epoch": 1737172913
        },
        "id": "551d03f8-680f-4a97-8c94-fb62bc55df40",
        "ownerId": "6df74633-edfc-4a1e-b49d-a51ed2220caa",
        "state": "Errored",
        "creationDate": {
            "date": "2025-01-18T04:01:49Z",
            "epoch": 1737172909
        },
        "type": "Content"
    },
    "eventType": "EntityErrored"
}
```

Will receive this JSON on finished entity event.

```
{
    "entity": {
        "contentType": "File",
        "c4id": "c421g7w2AWf1xCiP6vRgNU3iCtr65ZqVkT1DdrvC5RuCFK6syEVfrg24E3BKjELkPXKpBTGztervLkbUtegjcv9Gnt",
        "fileType": "Document",
        "originalDate": {
            "date": "2024-05-07T00:55:08Z",
            "epoch": 1715043308
        },
        "metadata": [{
                "name": "C4ID",
                "id": "e18af4bb-267e-4d2a-91a9-97fdbe88ffd3"
            }, {
                "name": "Document",
                "id": "ace98f3d-7794-4f69-aa59-1dbb64ec89ad"
            }
        ],
        "renditions": [{
                "name": "Master",
                "id": "8c3dcf6f-0fa5-4d34-9d98-875909551c9a"
            }, {
                "name": "Source",
                "id": "4583b569-3640-4f5b-b5e9-3a4e71d11c82"
            }, {
                "name": "Mezzanine",
                "id": "68dec874-0e8b-4213-9032-5776c1a3a149"
            }
        ],
        "links": [{
                "uri": "https://orcid.org/0009-0000-5827-7879",
                "linkType": "Web"
            }, {
                "uri": "https://orcid.org/0009-0009-4121-6880",
                "linkType": "Web"
            }, {
                "uri": "https://orcid.org/0009-0009-1570-3730",
                "linkType": "Web"
            }, {
                "uri": "https://orcid.org/0009-0009-5345-1773",
                "linkType": "Web"
            }, {
                "uri": "https://orcid.org/0009-0007-5670-2748",
                "linkType": "Web"
            }, {
                "uri": "https://orcid.org/0009-0009-5648-4183",
                "linkType": "Web"
            }, {
                "uri": "https://orcid.org/0009-0006-6892-3073",
                "linkType": "Web"
            }, {
                "uri": "https://doi.org/10.1145/3626772.3661370",
                "linkType": "Web"
            }, {
                "uri": "https://www.linkedin.com/company/linkedin/about/",
                "linkType": "LinkedIn"
            }, {
                "uri": "https://www.atlassian.com/software/jira",
                "linkType": "Web"
            }, {
                "uri": "https://qdrant.tech/",
                "linkType": "Web"
            }, {
                "uri": "https://doi.org/10.18653/v1/D18-1455",
                "linkType": "Web"
            }
        ],
        "mimeType": "application/pdf",
        "format": "PDF",
        "formatName": "Adobe Portable Document Format",
        "fileExtension": ".pdf",
        "fileName": "2404.17723v2.pdf",
        "fileSize": 794291,
        "masterUri": "https://graphlit20250109270424b0.blob.core.windows.net/files/c7e95daf-fd9e-4873-a13d-661595d7e5b4/2404.17723v2.pdf",
        "mezzanineUri": "https://graphlit20250109270424b0.blob.core.windows.net/files/c7e95daf-fd9e-4873-a13d-661595d7e5b4/Mezzanine/2404.17723v2.json",
        "textUri": "https://graphlit20250109270424b0.blob.core.windows.net/files/c7e95daf-fd9e-4873-a13d-661595d7e5b4/Mezzanine/2404.17723v2.json",
        "document": {
            "title": "Retrieval-Augmented Generation with Knowledge Graphs for Customer Service Question Answering",
            "subject": "-  Computing methodologies  ->  Information extraction.Natural language generation.",
            "software": "pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5",
            "pageCount": 5,
            "creationDate": "2024-05-07T00:55:08Z",
            "modifiedDate": "2024-05-07T00:55:08Z"
        },
        "workflowId": "1d741fed-38d5-4165-8035-20260e803492",
        "projectId": "d1d1a13d-3dba-463d-90bd-e6e78ab97274",
        "name": "2404.17723v2.pdf",
        "uri": "https://graphlitplatform.blob.core.windows.net/test/documents/2404.17723v2.pdf",
        "modifiedDate": {
            "date": "2025-01-18T04:06:20Z",
            "epoch": 1737173180
        },
        "startedDate": {
            "date": "2025-01-18T04:06:08Z",
            "epoch": 1737173168
        },
        "finishedDate": {
            "date": "2025-01-18T04:06:20Z",
            "epoch": 1737173180
        },
        "workflowDuration": {
            "time": "00:00:12.3250810",
            "ticks": 123250810
        },
        "id": "c7e95daf-fd9e-4873-a13d-661595d7e5b4",
        "ownerId": "d1d1a13d-3dba-463d-90bd-e6e78ab97274",
        "state": "Finished",
        "creationDate": {
            "date": "2025-01-18T04:06:07Z",
            "epoch": 1737173167
        },
        "type": "Content"
    },
    "eventType": "EntityFinished"
}
```
